# Data exploration 

In [ ]:
from __future__ import print_function

import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math as math
from scipy.stats import skew, norm, probplot

sns.set(rc={'figure.figsize':(11.7,6.27)})

In [ ]:
# Pipeline
from sklearn.base import TransformerMixin,BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline

# Preprocessing 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, PowerTransformer, OneHotEncoder

# Anamoly detection
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

# Model reduction
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

# Regressors 
from sklearn.linear_model import Ridge, ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge

# Model selection and validation
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Read Dat
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_test_id = df_test['Id']

# Remove ID
df_train = df_train.drop('Id',1)
df_test = df_test.drop('Id',1)


#MSSubClass=The building class
# df_train['MSSubClass'] = df_train['MSSubClass'].apply(str)
# df_train['MSSubClass'] = df_train['MSSubClass'].apply(str)

# #Changing OverallCond into a categorical variable
# df_train['OverallCond'] = df_train['OverallCond'].astype(str)
# df_test['OverallCond'] = df_test['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
# df_train['YrSold'] = df_train['YrSold'].astype(str)
# df_test['MoSold'] = df_test['MoSold'].astype(str)

# df_test['YrSold'] = df_test['YrSold'].astype(str)
# df_test['MoSold'] = df_test['MoSold'].astype(str)


In [ ]:
# find features
numeric_feats = df_train.dtypes[df_train.dtypes != "object"].index
categorical_feats = df_train.dtypes[df_train.dtypes == "object"].index

In [ ]:
df_train.head()

In [ ]:
df_train["SalePrice"].describe()

In [ ]:
sns.distplot(df_train["SalePrice"], kde=True)

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

In [ ]:
df_train.columns.to_series().groupby(df_train.dtypes).groups

In [ ]:
name= df_train.columns[7]
sns.scatterplot(x='TotalBsmtSF',y='SalePrice',data=df_train)

In [ ]:
sns.boxplot(x='OverallQual',y='SalePrice',data=df_train)

In [ ]:
g = sns.boxplot(x='Neighborhood',y='SalePrice',data=df_train)
g = g.set_xticklabels(g.get_xticklabels(), rotation=45)

In [ ]:
sns.set(rc={'figure.figsize':(30.7,6.27)})
g = sns.boxplot(x='YearBuilt',y='SalePrice',data=df_train)
g = g.set_xticklabels(g.get_xticklabels(), rotation=45)
sns.set(rc={'figure.figsize':(11.7,6.27)})

## Correlation Study

In [ ]:
#correlation matrix
#correlation matrix
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
corrmat.loc[corrmat['SalePrice']>0.5,'SalePrice'][:-1].sort_values(ascending=False)

Look at the features near diameter. TotalBsmtSF/FirstFlrSQ and GarageCars/GarageArea are highly correlated. 

In [ ]:
#scatterplot
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], height = 2.5)
plt.show();

# Transorm

## Anamoly Detection

In [ ]:
contamination = 0.005 # Ratio of contaminated data
anomaly_algorithms = {
    #"Robust covariance": EllipticEnvelope(contamination=contamination),
    "Isolation Forest": IsolationForest(behaviour='new', contamination=contamination,random_state=60),
    #"Local Outlier Factor": LocalOutlierFactor(contamination=contamination)        
}


variables = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt','LotArea']
outliers = dict()
for i, var in enumerate(['Total']):
    #val = df_train[var].values.reshape(-1,1)
    val = df_train[variables]
    for name, algorithm in anomaly_algorithms.items():
        # fit the data and tag outliers
        if name == "Local Outlier Factor":
            y_pred = algorithm.fit_predict(val)
        else:
            y_pred = algorithm.fit(val).predict(val)
                    
        outliers[var] = y_pred    

In [ ]:
# plot outliers
n_var = len(variables)
n_col = 4
fig, axarr = plt.subplots(math.ceil(n_var/n_col), n_col, figsize=(25, 8))
if (axarr.ndim==1):axarr = np.reshape(axarr, (-1, 2))     
for i, var in enumerate(variables):    
    outlier = pd.Series(outliers['Total']).map({1: 'normal', -1: 'outlier'}).astype('category')
    _ = sns.scatterplot(ax = axarr[i//n_col][i%n_col], x = df_train[var], y=df_train['SalePrice'], hue = outlier)

In [ ]:
# Remove outliers
df_train_clean = df_train.drop(index=np.where(outliers['Total']==-1)[0])

In [ ]:
np.where(outliers['Total']==-1)

## Missing Data

In [ ]:
total =df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data

In [ ]:
#dealing with missing data
df_train_tidy = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train_tidy = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train_tidy.isnull().sum().max() 

## Skweness

In [ ]:
#log transform skewed numeric features:
skewed_feats = df_train_tidy[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75].index
df_train_tidy[skewed_feats] = np.log1p(df_train_tidy[skewed_feats])

In [ ]:
sns.distplot(df_train_tidy['SalePrice'],fit=norm)
fig = plt.figure()
res = probplot(df_train['SalePrice'],plot=plt)

In [ ]:
sns.distplot(df_train_tidy['GrLivArea'],fit=norm)
fig = plt.figure()
res = probplot(df_train_tidy['GrLivArea'],plot=plt)

In [ ]:
sns.distplot(df_train['TotalBsmtSF'],fit=norm)
fig = plt.figure()
res = probplot(df_train['TotalBsmtSF'],plot=plt)

## Pre-processing pipelines

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)
            
#cs = ColumnSelector(columns=["state", "account length", "area code"])
#cs.fit_transform(df).head()

class SkewCorrection(BaseEstimator, TransformerMixin):
    def __init__(self, limit=0.75):
        self.limit = limit

    def fit(self, X, y=None):
        self.idx = skew(X)>self.limit
        return self

    def transform(self, X):
        X[:,self.idx] = np.log1p(X[:,self.idx]) # Using log1p to account for 0 inputs
        return X
    
    
class RemoveOutliers(BaseEstimator, TransformerMixin):
    def __init__(self, variables=['GrLivArea']):
        self.variables = variables

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X    

In [ ]:

pi = Pipeline(steps=[
    ('skew', SkewCorrection(limit=0.75))])
a = pi.fit_transform(df_train[numeric_feats])


# Tarining

## Numerical and categorical features

In [ ]:
# We create the preprocessing pipelines for both numeric and categorical data.
# Considered
# ('nonlinear', PowerTransformer(method='yeo-johnson', standardize=False)),
# ('pca',PCA())

numeric_features = numeric_feats.drop('SalePrice')#['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt','YearRemodAdd','GarageYrBlt','TotRmsAbvGrd']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('skew', SkewCorrection(limit=0.75)),
    ('scaler', RobustScaler())])

categorical_features = categorical_feats
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

## Estimation pipelines

In [ ]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', Ridge())])

x = df_train_clean.drop('SalePrice', axis=1)
y = df_train_clean['SalePrice']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

pipeline.fit(x_train, y_train)
print("model R2 score: %.3f" % pipeline.score(x_valid, y_valid))
print("RMSE: %.3f" % np.sqrt(mean_squared_error(y_valid,pipeline.predict(x_valid))))

kfold = 5
scoring = ['r2','neg_mean_squared_error']
results = cross_validate(pipeline, x, y, cv=kfold, return_train_score=True,scoring=scoring)
results


## Model Selection

In [ ]:
def model_cv(pipeline,x,y, kfold=5, scoring = ['r2','neg_mean_squared_error']):
    results = cross_validate(pipeline, x, y, cv=kfold, 
                             return_train_score=True, return_estimator=False, 
                             scoring=scoring,
                             n_jobs=-1) # Change the n_jobs to -1 to use all the cpus for calculation
    
    print('PL: {:<15} | test_score: {:1.4f} | train_score: {:1.4f} |\
    test_rmse: {:.4f} | fit_time: {:2.4f} | score_time: {:1.4f}'.format(name,
                                                                        np.mean(results['test_r2']),
                                                                        np.mean(results['train_r2']),
                                                                        np.sqrt(-np.mean(results['test_neg_mean_squared_error'])),                                                      
                                                                        np.mean(results['fit_time']),
                                                                        np.mean(results['score_time'])))  
    return True

regressors = {
    'Ridge': Ridge(),
    'Lasso': Lasso(alpha=0.1,max_iter=1000,tol=0.0001),
    'ElasticNet': ElasticNet(alpha=0.05, l1_ratio=.9, random_state=3),
    'KernelRidge': KernelRidge(),
    'GradBoost': GradientBoostingRegressor(),
    'RandForest': RandomForestRegressor()
}


for name, reg in regressors.items():
    steps=[('preprocessor', preprocessor),
           ('regressor', reg)]
    pipeline = Pipeline(steps)
    model_cv(pipeline,x,y)

print('Done!')    

## Hyper parameter tuning
We choose Gradient boosting algorithm here.

In [ ]:
steps=[('preprocessor', preprocessor),
       ('regressor', GradientBoostingRegressor())]
pipeline = Pipeline(steps)
pipeline.steps[1]

In [ ]:
regressors = {
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet(),
    'GradBoost': GradientBoostingRegressor(),
}
        
param_grid = {
    'Ridge':{
        'regressor__alpha': [0.001,0.01,0.1],
    },
    
    'ElasticNet':{
        'regressor__alpha': [0.001,0.01,0.1],
        'regressor__l1_ratio': [0.1,0.3,0.9],
        'regressor__random_state': [1,3,9]           
    },
    
    'GradBoost':{
        'preprocessor__num__imputer__strategy': ['mean', 'median'],    
        'regressor__alpha': [0.001,0.01,0.1],
        'regressor__n_estimators': [300,400],
        'regressor__max_depth': [3,6]        
    }
}

best_estimators = dict()

# Loop over specified estimators and find the best parameters
n_iter_search = 20
for name, reg in regressors.items():
    steps=[('preprocessor', preprocessor),
           ('regressor', reg)]
    pipeline = Pipeline(steps)
    # grid = GridSearchCV(pipeline, param_grid[name], cv=5, n_jobs=-1, iid=False)
    
    # The result in parameter settings is quite similar, while the run time for randomized search is drastically lower. 
    # The performance is slightly worse for the randomized search, though this is most likely a noise effect and would not carry over to a held-out test set.
    grid = RandomizedSearchCV(pipeline, param_grid[name], cv=5, n_jobs=-1, iid=False, n_iter=n_iter_search,)

    grid.fit(x,y)

    best_estimators[name] = grid.best_estimator_
    
    # summarize results
    print('================ Regressor: {:<10} ================'.format(name))
    print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
    means = grid.cv_results_['mean_test_score']
    stds = grid.cv_results_['std_test_score']
    params = grid.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

## Stacking the regressors

In [ ]:
predictions = pd.DataFrame()
for name, est in best_estimators.items():
    predictions[name]= est.predict(x)
    #model_cv(est,x,y)

In [ ]:
pipeline = Pipeline([('regressor', LinearRegression())])
    
model_cv(pipeline,predictions,y)
#stacked_estimator.fit(predictions, y)


In [ ]:
class StackedModel(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        predictions = pd.DataFrame()
        for name, est in best_estimators.items():
            predictions[name]= est.predict(X)
            
        self.pipeline = Pipeline([('regressor', LinearRegression())])
        self.pipeline.fit(predictions,y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = pd.DataFrame()
        for name, est in best_estimators.items():
            predictions[name]= est.predict(X)
            
        return self.pipeline.predict(predictions)
    
stackModel = StackedModel(best_estimators)    
stackModel.fit(x_train,y_train)
r2_score(y_valid,stackModel.predict(x_valid))

## Plot importatnt features

In [ ]:
# Plot important coefficients
coefs = pd.Series(clf.steps[1][1].coef_, index = X_train.columns)
print("Ridge picked " + str(sum(coefs != 0)) + " features and eliminated the other " +  \
      str(sum(coefs == 0)) + " features")
imp_coefs = pd.concat([coefs.sort_values().head(10),
                     coefs.sort_values().tail(10)])
imp_coefs.plot(kind = "barh")
plt.title("Coefficients in the Ridge Model")
plt.show()

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('nonlinear', PowerTransformer(method='yeo-johnson', standardize=True)),
    ('scaler', StandardScaler())])
pip1 = numeric_transformer.fit(df_train[['OverallQual', 'GrLivArea']], df_train['SalePrice'])
res = pip1.transform(df_train[['OverallQual', 'GrLivArea']])

In [ ]:
sns.distplot(df_train['TotalBsmtSF'],fit=norm)

In [ ]:
sns.distplot(res[:,1],fit=norm)

# Prediction

In [ ]:
y_pred = stackModel.predict(df_test)
y_pred

## Save Model

In [ ]:
from sklearn.externals import joblib 

joblib.dump(clf, 'my_regressor.pkl')

## Submission

In [ ]:
my_submission = pd.DataFrame({'Id': df_test_id, 'SalePrice': y_pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)